# GWAS with PLINK and R

## Installation
Optional: One could additionally install **HaploView** from https://sourceforge.net/projects/haploview/ and **gPLINK** from https://zzz.bwh.harvard.edu/plink/dist/gPLINK-2.050.zip

In [20]:
sudo apt install plink1.9

Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree... Done
Reading state information... 0% Reading state information... 0%Reading state information... Done
plink1.9 is already the newest version (1.90~b6.21-201019-1).
The following packages were automatically installed and are no longer required:
  dzen2 libfakekey0 libfltk1.3 libiw30 matchbox-common matchbox-desktop
  matchbox-keyboard matchbox-keyboard-im matchbox-panel matchbox-panel-manager
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.


In R install **qqman** with `install.packages("qqman")`

## Data

download example dataset by Shaun Purcell, however, it is also provided in this repository

In [1]:
# wget https://zzz.bwh.harvard.edu/plink/dist/example.zip

In [2]:
pwd

/home/ubuntu/CompBiol3/GWAS


In [3]:
ls -l

total 87288
drwxrwxr-x 2 ubuntu ubuntu     4096 Feb 25 10:49 Playground
drwxrwxr-x 3 ubuntu ubuntu     4096 Feb 25 10:09 ShaunPurcell
-rw-rw-r-- 1 ubuntu ubuntu     1064 Feb 25 10:49 gwas.ipynb
-rw-r--r-- 1 ubuntu ubuntu  7033003 Feb 25 10:52 gwas.org.map
-rw-r--r-- 1 ubuntu ubuntu 82332090 Feb 25 10:52 gwas.org.ped


## Processing

Initial creation of a binary dataset from raw genotype data for faster data access

In [87]:
plink1.9 --file gwas.org --make-bed --out gwas.bin

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
PLINK v1.90b6.22 64-bit (3 Nov 2020)           www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to gwas.bin.log.
Options in effect:
  --file gwas.org
  --make-bed
  --out gwas.bin

15983 MB RAM detected; reserving 7991 MB for main workspace.
.ped scan complete (for binary autoconversion).32425262728303132333435363738404142434445464748505152535455565758606162636465666768707172737475767778808182838485868788909192939495969798100%
Performing single-pass .bed write (228694 variants, 90 people).
--file: gwas.bin-temporary.bed + gwas.bin-temporary.bim +83940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394%
gwas.bin-temporary.fam written.
228694 variants loaded from .bim file.
90 people (45 males, 45 females) loaded from .fam.
90 phenotype values loaded from .fam.
Using 1 thread (

Check for missing called genotypes ...

In [72]:
awk '{allels+=NF-6;for(i=7; i<=NF; i++){if($i!~/[ATGC]/){mis++;allmis++}}print NR, NF-6, mis, (NF-6-mis)/(NF-6); n=0; mis=0}END{print allmis,allels,(allels-allmis)/(allels)}' gwas.org.ped | tail -4

88 457388 2130 0.995343
89 457388 15144 0.96689
90 457388 89608 0.804088
273896 41164920 0.993346


### Validate converted data

In [92]:
plink1.9 --maf 0.01 --geno 0.05 --mind 0.05 --write-snplist --make-bed --bfile gwas.bin --out gwas.bin.filtered

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
PLINK v1.90b6.22 64-bit (3 Nov 2020)           www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to gwas.bin.filtered.log.
Options in effect:
  --bfile gwas.bin
  --geno 0.05
  --maf 0.01
  --make-bed
  --mind 0.05
  --out gwas.bin.filtered
  --write-snplist

15983 MB RAM detected; reserving 7991 MB for main workspace.
228694 variants loaded from .bim file.
90 people (45 males, 45 females) loaded from .fam.
90 phenotype values loaded from .fam.
1 person removed due to missing genotype data (--mind).
ID written to gwas.bin.filtered.irem .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 89 founders and 0 nonfounders present.
Calculating allele frequencies... 1011121314151617181920212223242526272829303132333435363738394041424344454647484950515253545556575859606162636465666768697071727374757677787980

### GWAS

In [113]:
head -5 gwas.bin.fam

CH18526 NA18526 0 0 2 1
CH18524 NA18524 0 0 1 1
CH18529 NA18529 0 0 2 1
CH18558 NA18558 0 0 1 1
CH18532 NA18532 0 0 2 1


In [114]:
awk '{print $6}' gwas.bin.fam | sort | uniq -c

     41 1
     49 2


In [123]:
plink1.9 --bfile gwas.bin.filtered --assoc --adjust --out gwas.bin.filtered

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
PLINK v1.90b6.22 64-bit (3 Nov 2020)           www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to gwas.bin.filtered.log.
Options in effect:
  --adjust
  --assoc
  --bfile gwas.bin.filtered
  --out gwas.bin.filtered

15983 MB RAM detected; reserving 7991 MB for main workspace.
179562 variants loaded from .bim file.
89 people (44 males, 45 females) loaded from .fam.
89 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 89 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.996307.
179562 variants and 89 people pass filters and QC.
Among r

In [124]:
wc -l gwas.bin.filtered.assoc

179563 gwas.bin.filtered.assoc


In [125]:
head gwas.bin.filtered.assoc

 CHR         SNP         BP   A1      F_A      F_U   A2        CHISQ            P           OR 
   1   rs3094315     792429    G   0.1489  0.08537    A        1.684       0.1944        1.875 
   1   rs4040617     819185    G   0.1354  0.08537    A        1.111       0.2919        1.678 
   1   rs4075116    1043552    C  0.04167  0.07317    T       0.8278       0.3629       0.5507 
   1   rs9442385    1137258    T   0.3723   0.4268    G       0.5428       0.4613       0.7966 
   1  rs11260562    1205233    A  0.02174  0.03659    G       0.3424       0.5585       0.5852 
   1   rs6685064    1251215    C   0.3854    0.439    T       0.5253       0.4686       0.8013 
   1   rs3766180    1563420    T   0.1771  0.09756    C        2.317        0.128        1.991 
   1   rs6603791    1586208    A   0.1771  0.08537    G        3.189      0.07413        2.306 
   1   rs7519837    1596068    C   0.1702  0.08537    T        2.775      0.09573        2.198 


In [126]:
sort -g -k 9 gwas.bin.filtered.assoc | head -5

 CHR         SNP         BP   A1      F_A      F_U   A2        CHISQ            P           OR 
  11   rs2513514   75922141    A   0.5208   0.1585    G        25.39    4.693e-07        5.769 
  20   rs6110115   13911728    C   0.3085   0.6829    A        24.59    7.103e-07       0.2071 
  11   rs2508756   75921549    A   0.5417   0.1951    G         22.5    2.105e-06        4.875 
  15  rs16976702   54120691    G   0.5833   0.2317    C        22.43    2.183e-06        4.642 


In [127]:
awk '$9<0.05' gwas.bin.filtered.assoc | sort -g -k 9 | wc -l

13294


In [153]:
head -5 gwas.bin.filtered.assoc.adjusted

 CHR         SNP      UNADJ         GC       BONF       HOLM   SIDAK_SS   SIDAK_SD     FDR_BH     FDR_BY
  11   rs2513514  4.693e-07  7.131e-06    0.08427    0.08427    0.08081    0.08081    0.06378     0.8084 
  20   rs6110115  7.103e-07  9.938e-06     0.1276     0.1275     0.1198     0.1198    0.06378     0.8084 
  11   rs2508756  2.105e-06  2.373e-05      0.378     0.3779     0.3147     0.3147      0.098          1 
  15  rs16976702  2.183e-06  2.443e-05      0.392      0.392     0.3243     0.3243      0.098          1 


In [128]:
wc -l gwas.bin.filtered.assoc.adjusted 

179563 gwas.bin.filtered.assoc.adjusted


In [129]:
awk -v OFS="\t" '{print $1,$2,$3,$5,$9}' gwas.bin.filtered.assoc.adjusted | head -5

CHR	SNP	UNADJ	BONF	FDR_BH
11	rs2513514	4.693e-07	0.08427	0.06378
20	rs6110115	7.103e-07	0.1276	0.06378
11	rs2508756	2.105e-06	0.378	0.098
15	rs16976702	2.183e-06	0.392	0.098


In [130]:
awk -v OFS="\t" '$9<0.1{print $1,$2,$3,$5,$9}' gwas.bin.filtered.assoc.adjusted 

11	rs2513514	4.693e-07	0.08427	0.06378
20	rs6110115	7.103e-07	0.1276	0.06378
11	rs2508756	2.105e-06	0.378	0.098
15	rs16976702	2.183e-06	0.392	0.098


### Calculate Hardy-Weinberg

In [178]:
plink1.9 --bfile gwas.bin.filtered --hwe 0.00001 --make-bed --out gwas.bin.filtered.hwe

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
PLINK v1.90b6.22 64-bit (3 Nov 2020)           www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to gwas.bin.filtered.hwe.log.
Options in effect:
  --bfile gwas.bin.filtered
  --hwe 0.005
  --make-bed
  --out gwas.bin.filtered.hwe

15983 MB RAM detected; reserving 7991 MB for main workspace.
179562 variants loaded from .bim file.
89 people (44 males, 45 females) loaded from .fam.
89 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 89 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.996307.
--hwe: 358 variants removed due to Hardy-Wei

In [174]:
wc -l gwas.bin.filtered.hwe.bim gwas.bin.filtered.hwe.fam

 179562 gwas.bin.filtered.hwe.bim
     89 gwas.bin.filtered.hwe.fam
 179651 total


In [175]:
plink1.9 --bfile gwas.bin.filtered --hardy --out gwas.bin.filtered.hardy

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
PLINK v1.90b6.22 64-bit (3 Nov 2020)           www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to gwas.bin.filtered.hardy.log.
Options in effect:
  --bfile gwas.bin.filtered
  --hardy
  --out gwas.bin.filtered.hardy

15983 MB RAM detected; reserving 7991 MB for main workspace.
179562 variants loaded from .bim file.
89 people (44 males, 45 females) loaded from .fam.
89 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 89 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.996307.
--hardy: Writing Hardy-Weinberg report (founders only) to

In [176]:
head -5 gwas.bin.filtered.hardy.hwe

 CHR         SNP     TEST   A1   A2                 GENO   O(HET)   E(HET)            P 
   1   rs3094315      ALL    G    A              0/21/67   0.2386   0.2102       0.6015
   1   rs3094315      AFF    G    A              0/14/33   0.2979   0.2535       0.5702
   1   rs3094315    UNAFF    G    A               0/7/34   0.1707   0.1562            1
   1   rs4040617      ALL    G    A              0/20/69   0.2247   0.1995        0.592


Meaning of the columns:
gwas.bin.filtered.hwe.bim

In [140]:
awk '$3=="UNAFF" && $9<0.001' gwas.bin.filtered.hardy.hwe | head -5

   1  rs10399820    UNAFF    C    G               3/31/5   0.7949   0.4987    0.0003653
   2   rs2339516    UNAFF    A    G               5/5/31    0.122   0.2989    0.0009308
   2  rs17029230    UNAFF    A    G              12/9/19    0.225   0.4847    0.0008353
   2   rs3908093    UNAFF    A    G              11/8/21      0.2   0.4688    0.0004045
   2   rs6719884    UNAFF    C    A              14/9/18   0.2195   0.4952    0.0003949


In [145]:
egrep '(rs2513514|rs6110115|rs2508756|rs16976702).*UNAFF' gwas.bin.filtered.hardy.hwe

  11   rs2508756    UNAFF    A    G              1/14/26   0.3415   0.3141            1
  11   rs2513514    UNAFF    A    G              0/13/28   0.3171   0.2668       0.5649
  15  rs16976702    UNAFF    G    C              2/15/24   0.3659    0.356            1
  20   rs6110115    UNAFF    C    A              19/18/4    0.439   0.4331            1


### Filter Hardy
Filter for controls (TEXT=="UNAFF") and p-values <= 0.001

In [43]:
awk '$3=="UNAFF" && $9<=0.001{print $0}' hardy.hwe | sort -g -k 9 | head -3

  15   rs7178071    UNAFF    G    A               6/3/32  0.07317   0.2989    2.377e-05
  14   rs1242105    UNAFF    C    T               4/1/36  0.02439   0.1954    2.557e-05
   4   rs1392554    UNAFF    T    C              12/7/22   0.1707   0.4703    4.385e-05


### Create QC-filtered Dataset

In [46]:
plink1.9 --maf 0.01 --geno 0.05 --mind 0.05 --hwe 0.001 --bfile gwas.bin --make-bed --out gwas.bin.filtered

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [47]:
cat gwas.bin.filtered.log

PLINK v1.90b6.22 64-bit (3 Nov 2020)
Options in effect:
  --bfile gwas.bin
  --geno 0.05
  --gplink
  --hwe 0.001
  --maf 0.01
  --make-bed
  --mind 0.05
  --out gwas.bin.filtered

Hostname: rw20G
Working directory: /home/ubuntu/CompBiol3/GWAS
Start time: Mon Feb 26 11:16:07 2024

Random number seed: 1708942567
15983 MB RAM detected; reserving 7991 MB for main workspace.
228694 variants loaded from .bim file.
90 people (45 males, 45 females) loaded from .fam.
90 phenotype values loaded from .fam.
1 person removed due to missing genotype data (--mind).
ID written to gwas.bin.filtered.irem .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 89 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate in remaining samples is 0.995473.
2728 variants removed due to missing genotype data (--geno).
--hwe: 69 variants removed due to Hardy-Weinberg exact test.
46404 variants removed due to minor allele threshold(s)
(--ma

### Perform Basic Association Analysis of QC-Dataset

In [48]:
plink1.9 --bfile gwas.bin.filtered --assoc --adjust --out gwas.bin.filtered.assoc --gplink

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [49]:
cat gwas.bin.filtered.assoc.log

PLINK v1.90b6.22 64-bit (3 Nov 2020)
Options in effect:
  --adjust
  --assoc
  --bfile gwas.bin.filtered
  --gplink
  --out gwas.bin.filtered.assoc

Hostname: rw20G
Working directory: /home/ubuntu/CompBiol3/GWAS
Start time: Mon Feb 26 11:24:01 2024

Random number seed: 1708943041
15983 MB RAM detected; reserving 7991 MB for main workspace.
179493 variants loaded from .bim file.
89 people (44 males, 45 females) loaded from .fam.
89 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 89 founders and 0 nonfounders present.
Calculating allele frequencies... done.
Total genotyping rate is 0.996308.
179493 variants and 89 people pass filters and QC.
Among remaining phenotypes, 48 are cases and 41 are controls.
Writing C/C --assoc report to gwas.bin.filtered.assoc.assoc ...
done.
--adjust: Genomic inflation est. lambda (based on median chisq) = 1.25937.
--adjust values (179493 variants) written to
gwas.bin.filtered.assoc.asso

In [50]:
head -3 gwas.bin.filtered.assoc.assoc gwas.bin.filtered.assoc.assoc.adjusted

==> gwas.bin.filtered.assoc.assoc <==
 CHR         SNP         BP   A1      F_A      F_U   A2        CHISQ            P           OR 
   1   rs3094315     792429    G   0.1489  0.08537    A        1.684       0.1944        1.875 
   1   rs4040617     819185    G   0.1354  0.08537    A        1.111       0.2919        1.678 

==> gwas.bin.filtered.assoc.assoc.adjusted <==
 CHR         SNP      UNADJ         GC       BONF       HOLM   SIDAK_SS   SIDAK_SD     FDR_BH     FDR_BY
  11   rs2513514  4.693e-07  7.131e-06    0.08423    0.08423    0.08078    0.08078    0.06375      0.808 
  20   rs6110115  7.103e-07  9.938e-06     0.1275     0.1275     0.1197     0.1197    0.06375      0.808 


In [51]:
wc -l gwas.bin.filtered.assoc.assoc gwas.bin.filtered.assoc.assoc.adjusted

  179494 gwas.bin.filtered.assoc.assoc
  179494 gwas.bin.filtered.assoc.assoc.adjusted
  358988 total


### Analyze in R
```library(qqman)
gwas<-read.table("gwas.bin.filtered.assoc.assoc", header=TRUE)
manhattan(gwas, chr="CHR", bp="BP", snp="SNP", p="P",suggestiveline=F, genomewideline=T)
as.data.frame(table(gwas$CHR)) # number of SNPs per chromosome

SNPsOfInterest<-c("rs2513514","rs6110115")
manhattan(gwas, annotatePval = 0.01, highlight=SNPsOfInterest)
manhattan(gwas, annotatePval=0.001, highlight=SNPsOfInterest)
manhattan(gwas, annotatePval=0.0001, highlight=SNPsOfInterest)
manhattan(gwas, annotatePval=0.00001, highlight=SNPsOfInterest)
```

Fuse *gwas.bin.filtered.assoc.assoc* and *gwas.bin.filtered.assoc.assoc.adjusted* with respect to column "BP"

In [58]:
awk 'BEGIN{print "CHR","BP","SNP","P","GC","BONF"; while(getline < "gwas.bin.filtered.assoc.assoc" > 0){snp[$2]=$3}}{{print $1,snp[$2],$2,$3,$4,$5}}' gwas.bin.filtered.assoc.assoc.adjusted > gwas.bin.filtered.assoc.assoc.adjusted.fused

```
gwas_adj<-read.table("gwas.bin.filtered.assoc.assoc.adjusted.fused", header=TRUE)
manhattan(gwas_adj, chr="CHR", bp="BP", snp="SNP", p="P",suggestiveline=T, annotatePval=0.0001, genomewideline=T)
manhattan(subset(gwas_adj, CHR==8),xlim=c(12000000,14000000),genomewideline=T)
qq(gwas_adj$P)
```
Create plot with: `manhattan(gwas_adj, genomewideline=T, annotatePval=0.00001, annotateTop=F, highlight=gwas_adj$SNP[gwas_adj$BONF<1])`

![gwas.png](Images/gwas.png)

Create plot with: `manhattan(subset(gwas_adj, CHR==11),xlim=c(75000000,77000000),genomewideline=T, annotatePval=0.00001, annotateTop=F)`

![gwas-chr-11.png](Images/gwas-chr-11.png)

Create plot with: `manhattan(subset(gwas_adj, CHR==8),xlim=c(12000000,14000000),genomewideline=T)`

![gwas-chr-8.png](Images/gwas-chr-8.png)

Create plot with: `qq(gwas_adj$P)`

![gwas-qq.png](Images/gwas-qq.png)

In [53]:
grep rs9616985 gwas.bin.filtered.assoc.assoc.adjusted.fused

22 49519949 rs9616985 49519949 C 0.03125
22 49519949 rs9616985 0.8441 0.8609 1


In [55]:
tail -5 gwas.bin.filtered.assoc.assoc.adjusted.fused

21 33308533 rs2211685 1 1 1
21 39412266 rs2063972 1 1 1
21 42413116 rs220148 1 1 1
22 36001892 rs2267363 1 1 1
22 37662641 rs5757391 1 1 1


### Alternative Approach with gwaRs

In R install the package with `install.packages("gwaRs")`. 

In [147]:
R -e 'library("gwaRs")'

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k

R version 4.1.2 (2021-11-01) -- "Bird Hippie"
Copyright (C) 2021 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

[Previously saved workspace restored]

> library("gwaRs")
> 
> 



Fuse *gwas.bin.filtered.assoc* and *gwas.bin.filtered.assoc.adjusted* with respect to column "BP"

In [158]:
head -2 gwas.bin.filtered.assoc gwas.bin.filtered.assoc.adjusted

==> gwas.bin.filtered.assoc <==
 CHR         SNP         BP   A1      F_A      F_U   A2        CHISQ            P           OR 
   1   rs3094315     792429    G   0.1489  0.08537    A        1.684       0.1944        1.875 

==> gwas.bin.filtered.assoc.adjusted <==
 CHR         SNP      UNADJ         GC       BONF       HOLM   SIDAK_SS   SIDAK_SD     FDR_BH     FDR_BY
  11   rs2513514  4.693e-07  7.131e-06    0.08427    0.08427    0.08081    0.08081    0.06378     0.8084 


In [163]:
awk 'BEGIN{print "CHR","BP","SNP","P","GC","BONF"; while(getline < "gwas.bin.filtered.assoc" > 0){snp[$2]=$3}}$1!="CHR"{{print $1,snp[$2],$2,$3,$4,$5}}' gwas.bin.filtered.assoc.adjusted > gwas.bin.filtered.assoc.adjusted.fused

In [168]:
head  gwas.bin.filtered.assoc.adjusted.fused

CHR BP SNP P GC BONF
11 75922141 rs2513514 4.693e-07 7.131e-06 0.08427
20 13911728 rs6110115 7.103e-07 9.938e-06 0.1276
11 75921549 rs2508756 2.105e-06 2.373e-05 0.378
15 54120691 rs16976702 2.183e-06 2.443e-05 0.392
8 12895576 rs11204005 7.882e-06 6.841e-05 1
9 94478347 rs16910850 1.216e-05 9.688e-05 1
12 129970575 rs1195747 1.427e-05 0.0001102 1
17 77933018 rs7207095 1.682e-05 0.0001257 1
15 77672467 rs16971118 1.907e-05 0.0001391 1


In [169]:
# Create Manhattan Plot
R -e 'png("Images/man-plot.png", width = 1000, height = 600);
library("gwaRs");
gwas_adj<-read.table("gwas.bin.filtered.assoc.adjusted.fused", header=TRUE);
man_plot(gwas_adj, annotatePval=0.00001, highlight=gwas_adj$SNP[gwas_adj$BONF<1], chromCol=c("tomato","lightslateblue"));
dev.off()'

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k

R version 4.1.2 (2021-11-01) -- "Bird Hippie"
Copyright (C) 2021 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

[Previously saved workspace restored]

> png("Images/man-plot.png", width = 1000, height = 600);
> library("gwaRs");
> gwas_adj<-read.table("gwas.bin.filtered.assoc.adjusted.fused", header=TRUE);
> man_plot(gwas_adj, annotatePval=0.00001, highlight=gwas_adj$SNP[gwas_adj$BONF<1], chromCo

![Manhattan Plot](Images/man-plot.png)

In [170]:
# Create Q-Q Plot
R -e 'png("Images/qq-plot.png", width = 1000, height = 600);
library("gwaRs");
gwas_adj<-read.table("gwas.bin.filtered.assoc.adjusted.fused", header=TRUE);
qq_plot(gwas_adj);
dev.off()'

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k

R version 4.1.2 (2021-11-01) -- "Bird Hippie"
Copyright (C) 2021 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

[Previously saved workspace restored]

> png("Images/qq-plot.png", width = 1000, height = 600);
> library("gwaRs");
> gwas_adj<-read.table("gwas.bin.filtered.assoc.adjusted.fused", header=TRUE);
> qq_plot(gwas_adj);
> dev.off()
null device 
          1 
> 
> 



![qq-plot](Images/qq-plot.png)

In [171]:
# Create Karyotype Plot
R -e 'png("Images/karyo-plot.png", width = 1000, height = 600);
library("gwaRs");
gwas_adj<-read.table("gwas.bin.filtered.assoc.adjusted.fused", header=TRUE);
karyotype_plot(gwas_adj);
dev.off()'

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k

R version 4.1.2 (2021-11-01) -- "Bird Hippie"
Copyright (C) 2021 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

[Previously saved workspace restored]

> png("Images/karyo-plot.png", width = 1000, height = 600);
> library("gwaRs");
> gwas_adj<-read.table("gwas.bin.filtered.assoc.adjusted.fused", header=TRUE);
> karyotype_plot(gwas_adj);
> dev.off()
null device 
          1 
> 
> 



![karyo-plot](Images/karyo-plot.png)